# Homelessness Predictor Based on City Budgets

The purpose of this project is to see if we can predict homelessness by city from the city's budget. The hypothesis here is that something in the way that a city decides to spend its funds will give an insight to a model on how to predict number of homeless.

In [164]:
import pandas as pd
import numpy as np



In [165]:
hl = pd.read_csv('Homelessness_Count_-_USA.csv')
fisc = pd.read_csv('fisc_data.csv')

In [166]:
hl.head(2)

,CoC Number,CoC Name,Year,Attribute Name,Value,Year - Text
0,AK-500,Anchorage CoC,2014,Chronically Homeless,101,2014
1,AK-500,Anchorage CoC,2014,Sheltered Homeless People in Families,282,2014


In [167]:
hl.shape

(25515, 6)

In [168]:
fisc.head(2)

,year,city_name,city_types,id_city,city_population,cpi,rev_total_city,rev_general_city,intergovt_rev_city,igr_federal_city,...,cash_other_offsets,cash_other_bonds,cash_other_other,county_name,id_county,county_population,relationship_city_school,enrollment,districts_in_city,consolidated_govt
0,1977,AK: Anchorage,core,22002001,174500,4.044885,5342.24,4956.92,2148.77,279.32,...,178.51,787.93,691.32,NaN,NaN,NaN,4.0,36855.0,NaN,1.0
1,1978,AK: Anchorage,core,22002001,177000,3.759509,5948.99,5490.05,2468.11,403.24,...,187.53,1395.82,1158.01,NaN,NaN,NaN,4.0,36804.0,NaN,1.0


In [169]:
fisc.shape

(8938, 663)

In [170]:
fisc_2015 = fisc[fisc['year'] == 2015]
fisc_2015.head(2)

,year,city_name,city_types,id_city,city_population,cpi,rev_total_city,rev_general_city,intergovt_rev_city,igr_federal_city,...,cash_other_offsets,cash_other_bonds,cash_other_other,county_name,id_county,county_population,relationship_city_school,enrollment,districts_in_city,consolidated_govt
38,2015,AK: Anchorage,core,22002001,299927,1.034187,6054.78,5276.45,2469.25,152.67,...,191.48,0.00,2652.64,NaN,NaN,NaN,4.0,48368.0,NaN,1.0
79,2015,AK: Fairbanks,core,22006001,31529,1.034187,1891.35,1891.35,678.13,42.02,...,113.06,1.93,5897.51,Fairbanks North Star,21006002.0,99256.0,5.0,4716.0,NaN,0.0


In [171]:
# create separate State column
fisc_2015['state'] = fisc_2015['city_name'].astype(str).str[:2]
fisc_2015.head(2)

/var/folders/vs/w7rl81xx4v3dmfdm5sty1pl40000gn/T/ipykernel_18125/2675751203.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fisc_2015['state'] = fisc_2015['city_name'].astype(str).str[:2]


,year,city_name,city_types,id_city,city_population,cpi,rev_total_city,rev_general_city,intergovt_rev_city,igr_federal_city,...,cash_other_bonds,cash_other_other,county_name,id_county,county_population,relationship_city_school,enrollment,districts_in_city,consolidated_govt,state
38,2015,AK: Anchorage,core,22002001,299927,1.034187,6054.78,5276.45,2469.25,152.67,...,0.00,2652.64,NaN,NaN,NaN,4.0,48368.0,NaN,1.0,AK
79,2015,AK: Fairbanks,core,22006001,31529,1.034187,1891.35,1891.35,678.13,42.02,...,1.93,5897.51,Fairbanks North Star,21006002.0,99256.0,5.0,4716.0,NaN,0.0,AK


In [173]:
fisc_2015['city'] = fisc_2015['city_name'].astype(str).str[4:]
fisc_2015.head()

/var/folders/vs/w7rl81xx4v3dmfdm5sty1pl40000gn/T/ipykernel_18125/1070362349.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fisc_2015['city'] = fisc_2015['city_name'].astype(str).str[4:]


,year,city_name,city_types,id_city,city_population,cpi,rev_total_city,rev_general_city,intergovt_rev_city,igr_federal_city,...,cash_other_other,county_name,id_county,county_population,relationship_city_school,enrollment,districts_in_city,consolidated_govt,state,city
38,2015,AK: Anchorage,core,22002001,299927,1.034187,6054.78,5276.45,2469.25,152.67,...,2652.64,NaN,NaN,NaN,4.0,48368.0,NaN,1.0,AK,Anchorage
79,2015,AK: Fairbanks,core,22006001,31529,1.034187,1891.35,1891.35,678.13,42.02,...,5897.51,Fairbanks North Star,21006002.0,99256.0,5.0,4716.0,NaN,0.0,AK,Fairbanks
120,2015,AL: Birmingham,core|legacy,12037003,211109,1.034187,3426.90,2369.36,193.36,46.07,...,2721.32,Jefferson,11037037.0,659918.0,1.0,24693.0,1.0,0.0,AL,Birmingham
161,2015,AL: Gadsden,legacy,12028003,36057,1.034187,2063.56,1785.14,57.79,42.79,...,1205.09,Etowah,11028028.0,103407.0,1.0,5226.0,1.0,0.0,AL,Gadsden
202,2015,AL: Mobile,core,12049004,192979,1.034187,1984.90,1693.66,16.89,11.81,...,1592.10,Mobile,11049049.0,414590.0,2.0,25673.0,1.0,0.0,AL,Mobile


In [174]:
fisc_2015['state'].unique()

array(['AK', 'AL', 'AR', 'AZ', 'Av', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL',
       'GA', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
       'MI', 'MN', 'MO', 'MS', 'MT', 'Me', 'NC', 'ND', 'NE', 'NH', 'NJ',
       'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN',
       'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'], dtype=object)

In [175]:
hl['Year'].value_counts()

2015    17052
2014     8463
Name: Year, dtype: int64

In [176]:
hl['CoC Name'].value_counts().sort_values(ascending=False)

Anchorage CoC                                           63
Fayetteville/Northwest Arkansas CoC                     63
Birmingham/Jefferson, St. Clair, Shelby Counties CoC    63
Mobile City & County/Baldwin County CoC                 63
Florence/Northwest Alabama CoC                          63
                                                        ..
Delta Hills CoC                                         42
Tehama County CoC                                       42
Lake County CoC                                         42
Inyo, Mono, Alpine Counties CoC                         42
Total                                                   21
Name: CoC Name, Length: 407, dtype: int64

In [178]:
len(hl['CoC Name'].unique())

407

In [179]:
len(fisc_2015['city_name'].unique())

218

In [192]:
hl[hl['Attribute Name'] == 'Homeless Individuals']['Value'].sum()

1078800

In [181]:
hl_2015 = hl[hl['Year'] == 2015]

In [182]:
hl_2015.head(2)

,CoC Number,CoC Name,Year,Attribute Name,Value,Year - Text
98,NJ-512,Salem County CoC,2015,Homeless Individuals,18,2015
199,NJ-512,Salem County CoC,2015,Sheltered Homeless,37,2015


In [183]:
# focus just on count of homeless individuals
hl_2015_ind = hl_2015[hl_2015['Attribute Name'] == 'Homeless Individuals']
hl_2015_ind

,CoC Number,CoC Name,Year,Attribute Name,Value,Year - Text
98,NJ-512,Salem County CoC,2015,Homeless Individuals,18,2015
8494,AK-500,Anchorage CoC,2015,Homeless Individuals,837,2015
8537,AK-501,Alaska Balance of State CoC,2015,Homeless Individuals,505,2015
8580,AL-500,"Birmingham/Jefferson, St. Clair, Shelby Counti...",2015,Homeless Individuals,886,2015
8622,AL-501,Mobile City & County/Baldwin County CoC,2015,Homeless Individuals,470,2015
...,...,...,...,...,...,...
25331,WV-500,Wheeling/Weirton Area CoC,2015,Homeless Individuals,79,2015
25373,WV-501,"Huntington/Cabell, Wayne Counties CoC",2015,Homeless Individuals,123,2015
25415,WV-503,"Charleston/Kanawha, Putnam, Boone, Clay Counti...",2015,Homeless Individuals,323,2015
25457,WV-508,West Virginia Balance of State CoC,2015,Homeless Individuals,889,2015


In [184]:
# remove state rows from hl
hl_cities = hl_2015_ind[~hl_2015_ind['CoC Name'].str.contains('State')]

In [193]:
hl_cities.head(2)

,CoC Number,CoC Name,Year,Attribute Name,Value,Year - Text
98,NJ-512,Salem County CoC,2015,Homeless Individuals,18,2015
8494,AK-500,Anchorage CoC,2015,Homeless Individuals,837,2015


In [237]:
hl_cities.drop(columns='Year - Text', inplace=True)

/var/folders/vs/w7rl81xx4v3dmfdm5sty1pl40000gn/T/ipykernel_18125/3046618538.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hl_cities.drop(columns='Year - Text', inplace=True)


### Individual City Name Cleanup for Merge

In [220]:
hl_cities['CoC Name']

98                                        Salem County CoC
8494                                         Anchorage CoC
8580     Birmingham/Jefferson, St. Clair, Shelby Counti...
8622               Mobile City & County/Baldwin County CoC
8664                        Florence/Northwest Alabama CoC
                               ...                        
25247                             Racine City & County CoC
25289                              Madison/Dane County CoC
25331                            Wheeling/Weirton Area CoC
25373                Huntington/Cabell, Wayne Counties CoC
25415    Charleston/Kanawha, Putnam, Boone, Clay Counti...
Name: CoC Name, Length: 366, dtype: object

In [223]:
# remove CoC from city names
hl_cities['CoC Name'] = hl_cities['CoC Name'].str.replace(' CoC', '')
hl_cities.head(2)

/var/folders/vs/w7rl81xx4v3dmfdm5sty1pl40000gn/T/ipykernel_18125/2507037332.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hl_cities['CoC Name'] = hl_cities['CoC Name'].str.replace(' CoC', '')


,CoC Number,CoC Name,Year,Attribute Name,Value,Year - Text
98,NJ-512,Salem County,2015,Homeless Individuals,18,2015
8494,AK-500,Anchorage,2015,Homeless Individuals,837,2015


In [226]:
# remove atlanta's "cont of care" 
hl_cities['CoC Name'] = hl_cities['CoC Name'].str.replace(' Continuum of Care', '')

/var/folders/vs/w7rl81xx4v3dmfdm5sty1pl40000gn/T/ipykernel_18125/3351277880.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hl_cities['CoC Name'] = hl_cities['CoC Name'].str.replace(' Continuum of Care', '')


In [234]:
# change district of columbia to washington
hl_cities['CoC Name'] = hl_cities['CoC Name'].str.replace('District of Columbia', 'Washington')

/var/folders/vs/w7rl81xx4v3dmfdm5sty1pl40000gn/T/ipykernel_18125/2154841501.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hl_cities['CoC Name'] = hl_cities['CoC Name'].str.replace('District of Columbia', 'Washington')


In [ ]:
hl_cities['CoC Name'] = hl_cities['CoC Name'].str.replace('District of Columbia', 'Washington')
hl_cities['CoC Name'] = hl_cities['CoC Name'].str.replace('New York City', 'New York')
hl_cities['CoC Name'] = hl_cities['CoC Name'].str.replace('Metropolitan Denver Homeless Initiative', 'Denver')
hl_cities['CoC Name'] = hl_cities['CoC Name'].str.replace('Oklahoma City', 'Oklahoma')
hl_cities['CoC Name'] = hl_cities['CoC Name'].str.replace('St.Louis City', 'St. Louis')
hl_cities['CoC Name'] = hl_cities['CoC Name'].str.replace('Salt Lake City & County', 'Salt Lake City')
hl_cities['CoC Name'] = hl_cities['CoC Name'].str.replace('Baltimore City', 'Baltimore')

In [266]:
# springfield MA, OH and portland OR, ME issue - duplicate names
#hl_cities['CoC Name'] = hl_cities['CoC Name'].str.replace('Springfield', 'Springfield MA')
fisc_2015[fisc_2015['city'] == 'Springfield']

,year,city_name,city_types,id_city,city_population,cpi,rev_total_city,rev_general_city,intergovt_rev_city,igr_federal_city,...,cash_other_other,county_name,id_county,county_population,relationship_city_school,enrollment,districts_in_city,consolidated_govt,state,city
3482,2015,MA: Springfield,core,222007003,154513,1.034187,5117.45,4814.07,3278.09,197.18,...,1168.46,Hampden,NaN,469507.0,4.0,28167.0,NaN,1.0,MA,Springfield
6475,2015,OH: Springfield,legacy,362012008,59730,1.034187,1343.16,1221.68,225.09,124.23,...,1783.37,Clark,361012012.0,136316.0,3.0,8476.0,3.0,0.0,OH,Springfield


In [256]:
hl_cities[hl_cities['CoC Name'] == 'Salt Lake City']

,CoC Number,CoC Name,Year,Attribute Name,Value
23947,UT-500,Salt Lake City,2015,Homeless Individuals,1346


In [186]:
# remove Average rows from fisc
fisc_2015 = fisc_2015[~fisc_2015['city_name'].str.contains('Average')]

In [187]:
fisc_2015.shape

(215, 665)

In [189]:
# which columns of fisc to use? 

In [190]:
fisc_2015.head(1)

,year,city_name,city_types,id_city,city_population,cpi,rev_total_city,rev_general_city,intergovt_rev_city,igr_federal_city,...,cash_other_other,county_name,id_county,county_population,relationship_city_school,enrollment,districts_in_city,consolidated_govt,state,city
38,2015,AK: Anchorage,core,22002001,299927,1.034187,6054.78,5276.45,2469.25,152.67,...,2652.64,NaN,NaN,NaN,4.0,48368.0,NaN,1.0,AK,Anchorage


In [239]:
#export cleaned dfs
fisc_2015.to_csv('fisc_2015_cities_only.csv', index=False)
hl_cities.to_csv('hl_cities.csv', index=False)

In [195]:
fisc_2015[fisc_2015['city'] == 'Fairbanks']['rev_total_city']

79    1891.35
Name: rev_total_city, dtype: float64

In [196]:
fisc_2015['rev_total_city'].describe()

count      215.000000
mean      3079.799070
std       2206.414711
min        781.840000
25%       1694.150000
50%       2285.490000
75%       4050.175000
max      20433.440000
Name: rev_total_city, dtype: float64

## Adding together datasets

In [225]:
hl_cities

,CoC Number,CoC Name,Year,Attribute Name,Value,Year - Text
98,NJ-512,Salem County,2015,Homeless Individuals,18,2015
8494,AK-500,Anchorage,2015,Homeless Individuals,837,2015
8580,AL-500,"Birmingham/Jefferson, St. Clair, Shelby Counties",2015,Homeless Individuals,886,2015
8622,AL-501,Mobile City & County/Baldwin County,2015,Homeless Individuals,470,2015
8664,AL-502,Florence/Northwest Alabama,2015,Homeless Individuals,158,2015
...,...,...,...,...,...,...
25247,WI-502,Racine City & County,2015,Homeless Individuals,109,2015
25289,WI-503,Madison/Dane County,2015,Homeless Individuals,449,2015
25331,WV-500,Wheeling/Weirton Area,2015,Homeless Individuals,79,2015
25373,WV-501,"Huntington/Cabell, Wayne Counties",2015,Homeless Individuals,123,2015


In [267]:
new_df = pd.merge(left=hl_cities, right=fisc_2015, left_on='CoC Name', right_on='city')

In [268]:
new_df.sort_values(by='Value', ascending=False)

,CoC Number,CoC Name,Year,Attribute Name,Value,year,city_name,city_types,id_city,city_population,...,cash_other_other,county_name,id_county,county_population,relationship_city_school,enrollment,districts_in_city,consolidated_govt,state,city
24,NY-600,New York,2015,Homeless Individuals,29612,2015,NY: New York,core,332031001,8437387,...,2607.01,NaN,NaN,NaN,4.0,1051594.0,NaN,1.0,NY,New York
1,CA-501,San Francisco,2015,Homeless Individuals,6175,2015,CA: San Francisco,core,52038001,851116,...,8962.01,NaN,NaN,NaN,1.0,58865.0,1.0,1.0,CA,San Francisco
7,IL-510,Chicago,2015,Homeless Individuals,4592,2015,IL: Chicago,core|legacy,142016016,2728524,...,2055.60,Cook,141016016.0,5257481.0,1.0,387311.0,1.0,0.0,IL,Chicago
5,DC-500,Washington,2015,Homeless Individuals,3821,2015,DC: Washington,core|legacy,92001001,662513,...,6268.93,NaN,NaN,NaN,4.0,61228.0,NaN,1.0,DC,Washington
6,GA-500,Atlanta,2015,Homeless Individuals,3752,2015,GA: Atlanta,core|legacy,112060002,460998,...,7346.45,Fulton,111060060.0,992022.0,1.0,51500.0,1.0,0.0,GA,Atlanta
26,PA-500,Philadelphia,2015,Homeless Individuals,3346,2015,PA: Philadelphia,core|legacy,392051001,1565604,...,1884.95,NaN,NaN,NaN,1.0,166233.0,1.0,1.0,PA,Philadelphia
3,CO-503,Denver,2015,Homeless Individuals,3039,2015,CO: Denver,core,62016001,664870,...,2750.43,NaN,NaN,NaN,1.0,69766.0,1.0,1.0,CO,Denver
10,MA-500,Boston,2015,Homeless Individuals,2539,2015,MA: Boston,core|legacy,222013001,663017,...,2482.82,Suffolk,NaN,773839.0,4.0,66905.0,NaN,1.0,MA,Boston
16,MD-501,Baltimore,2015,Homeless Individuals,2193,2015,MD: Baltimore,core|legacy,212004001,623165,...,2539.07,NaN,NaN,NaN,4.0,78384.0,NaN,1.0,MD,Baltimore
19,MI-501,Detroit,2015,Homeless Individuals,1964,2015,MI: Detroit,core|legacy,232082004,682669,...,2514.96,Wayne,231082082.0,1771931.0,1.0,74365.0,1.0,0.0,MI,Detroit


In [269]:
new_df.to_csv('hl_fisc.csv', index=False)